In [ ]:
def getKmers(sequence, size, step): 
    output_seq=[]
    for x in range(0, len(sequence) - size, step):
        output_seq.append(sequence[x:x+size])
    return(output_seq)

In [ ]:
parameters = {
    #    "loss":["deviance"],
    #"learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    #"min_samples_split": np.linspace(0.1, 0.5, 12),
    #"min_samples_leaf": np.linspace(0.1, 0.5, 12),
        "max_depth":[1,3,5],
    #"max_features":["log2","sqrt"],
    #"criterion": ["friedman_mse",  "mae"],
        "subsample":[ 0.8, 1.0],
        "n_estimators":[5,10]
            }

clf = GridSearchCV(GradientBoostingRegressor(), parameters, cv=3, n_jobs=-1)

clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.best_params_)




unique_sequences_freq

In [ ]:
df_imporatnce=pd.DataFrame({"un":unique_sequences, "importance":clf.best_estimator_.feature_importances_})
df_imporatnce=df_imporatnce.loc[df_imporatnce['importance'] > 0]
df_imporatnce

In [ ]:
import matplotlib.pyplot as mp
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor
from sklearn import datasets
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler

import numpy as np 

result_f=[]
label_index=2
for k in range(2,4):
    import pandas as pd
    df_input=pd.read_csv("/Users/moradigd/Documents/Plasmid/input_agg.csv")
    df_input.columns
    df_input=df_input.iloc[:,[0,k]]
    df_input.shape
    df_input=df_input.dropna()
    df_input=df_input.reset_index()

     
    for g in range(9,10):
        
        print("start")
        unique_sequences_set=[]
#for i in range(len(df_input['seq'])):

        for i in range(df_input.shape[0]):
            print(i)
            unique_sequences_set=unique_sequences_set+getKmers(df_input['seq'][i], g, 1)
        unique_sequences=list(set(unique_sequences_set))

        print("done")
#"abcdabcva".count("ab")
#df_input['seq'][1].count(unique_sequences[1])

        [df_input['seq'][1].count(i) for i in unique_sequences]
        
        print("start")
        unique_sequences_freq=[]
#for i in range(len(df_input['seq'])):
        for j in range(df_input.shape[0]):
            print(j)
            unique_sequences_freq.append([df_input['seq'][j].count(i) for i in unique_sequences])
        unique_sequences_freq   
        unique_sequences_freq=pd.DataFrame(unique_sequences_freq)
#unique_sequences_freq.columns=unique_sequences
        unique_sequences_freq
        print("done")


        unique_sequences_freq[['label']]=df_input.iloc[0:df_input.shape[0],label_index]
        unique_sequences_freq
        
        scaler = MinMaxScaler()
        unique_sequences_freq = pd.DataFrame(scaler.fit_transform(unique_sequences_freq), columns=unique_sequences_freq.columns)

        data=unique_sequences_freq.values

        X, y = data[:, :-1], data[:, -1]
        
        
        for m in range(10):
        
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
    
            parameters = {
    #    "loss":["deviance"],
    #"learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    #"min_samples_split": np.linspace(0.1, 0.5, 12),
    #"min_samples_leaf": np.linspace(0.1, 0.5, 12),
        "max_depth":[1,3,5],
    #"max_features":["log2","sqrt"],
    #"criterion": ["friedman_mse",  "mae"],
        "subsample":[ 0.8, 1.0],
        "n_estimators":[5,10]
            }

            clf = GridSearchCV(GradientBoostingRegressor(), parameters, cv=3, n_jobs=-1)

            clf.fit(X_train, y_train)
            print(clf.score(X_train, y_train))
            print(clf.best_params_)

        #correct_test = correct_data(test)
        #testX = correct_test[predictor].values
            result = clf.predict(X_test)
            
            df_imporatnce=pd.DataFrame({"tag":unique_sequences, "importance":clf.best_estimator_.feature_importances_})
            df_imporatnce=df_imporatnce.loc[df_imporatnce['importance'] > 0]
            df_imporatnce.to_csv("/Users/moradigd/Documents/Plasmid/importance"+str(g)+'_'+str(m)+'_'+str(df_input.columns[2]))
            print(df_imporatnce)

    
            from scipy import stats
            rho, pval = stats.spearmanr(result, y_test)
            print(rho)
            print(pval)
            print(df_input.shape)
            print(df_input.columns)
            result_f.append([rho,pval, g, df_input.columns[2], 'gb',m])
        
        
        
            from sklearn.linear_model import LassoCV
        #reg = LassoCV(cv=3, random_state=0).fit(X_train, y_train)
        
            lasso = Lasso(random_state=0, max_iter=10000)
            alphas = np.logspace(-4, -0.5, 10)
    
            tuned_parameters = [{'alpha': alphas}]
            clf = GridSearchCV(lasso, tuned_parameters, cv=3)
            clf.fit(X_train, y_train)
            print(clf.best_params_)
            result = clf.predict(X_test)

    
            from scipy import stats
            rho, pval = stats.spearmanr(result, y_test)
            print(rho)
            print(pval)
            print(df_input.shape)
            print(df_input.columns)
            result_f.append([rho,pval, g, df_input.columns[2], 'lasso',m])
            
            pd.DataFrame(result_f).to_csv("/Users/moradigd/Documents/Plasmid/results_screening_agg_scale.csv")
        
        #model = XGBRegressor()
        #model.fit(X[0:int(df_input.shape[0]*0.8) , :] , y[0:int(df_input.shape[0]*0.8) ])
        #model.predict(X[int(df_input.shape[0]*0.8) :df_input.shape[0], :])

        #mp.scatter(model.predict(X[int(df_input.shape[0]*0.8) :df_input.shape[0], :]), y[int(df_input.shape[0]*0.8) :df_input.shape[0]])
        #mp.xlabel('array1')
        #mp.ylabel('array2')
        #from scipy import stats
        #rho, pval = stats.spearmanr(model.predict(X[int(df_input.shape[0]*0.8) :df_input.shape[0], :]), y[int(df_input.shape[0]*0.8) :df_input.shape[0]])
        #print(rho)
        #print(pval)
        #print(df_input.shape)
        #print(df_input.columns)
        #result.append([rho,pval, g, df_input.columns[2]])


In [ ]:
data.shape

In [ ]:
unique_sequences_freq.to_csv("/Users/moradigd/Documents/Plasmid/interval.csv")

In [ ]:
unique_sequences_freq

In [ ]:
import matplotlib.pyplot as mp
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor
from sklearn import datasets
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler

import numpy as np 

result_f=[]
label_index=2
for k in range(2,4):
    import pandas as pd
    df_input=pd.read_csv("/Users/moradigd/Documents/Plasmid/input_agg.csv")
    df_input.columns
    df_input=df_input.iloc[:,[0,k]]
    df_input.shape
    df_input=df_input.dropna()
    df_input=df_input.reset_index()
    unique_sequences_agg=[]
    df_sequences_agg= pd.DataFrame()
     
    for g in range(5,7):
        
        print("start")
        unique_sequences_set=[]
#for i in range(len(df_input['seq'])):

        for i in range(df_input.shape[0]):
            unique_sequences_set=unique_sequences_set+getKmers(df_input['seq'][i], g, 1)
        unique_sequences=list(set(unique_sequences_set))
        unique_sequences_agg=unique_sequences_agg+unique_sequences

        print("done")
#"abcdabcva".count("ab")
#df_input['seq'][1].count(unique_sequences[1])

        [df_input['seq'][1].count(i) for i in unique_sequences]
        
        print("start")
        unique_sequences_freq=[]
#for i in range(len(df_input['seq'])):
        for j in range(df_input.shape[0]):
            unique_sequences_freq.append([df_input['seq'][j].count(i) for i in unique_sequences])
        unique_sequences_freq   
        unique_sequences_freq=pd.DataFrame(unique_sequences_freq)
#unique_sequences_freq.columns=unique_sequences
        unique_sequences_freq
    
        #df_sequences_agg= pd.concat([df_sequences_agg,unique_sequences_freq])
        #InfoDF = pd.concat([InfoDF,tempDF])
        print("done")

In [ ]:
len(unique_sequences_agg)
#len(unique_sequences)
df_sequences_agg.shape
#unique_sequences_freq.shape
df_sequences_agg

In [ ]:
df_input

In [ ]:
unique_sequences_freq.shape

In [ ]:
unique_sequences_freq_t=unique_sequences_freq.transpose()

In [ ]:
unique_sequences_freq_t.columns=['seq'+str(i) for i in range(unique_sequences_freq_t.shape[1])]

In [ ]:
unique_sequences_freq_t.index=unique_sequences

In [ ]:
unique_sequences_freq_t

In [ ]:
np.zeros((unique_sequences_freq_t.shape[0],))

In [ ]:
labels=["Non-unique Gene name","Annotation","No. isolates","No. sequences","Avg sequences per isolate","Genome Fragment","Order within Fragment","Accessory Fragment","Accessory Order with Fragment","QC","Min group size nuc","Max group size nuc","Avg group size nuc"]
unique_sequences_freq_left=pd.DataFrame(np.zeros((unique_sequences_freq_t.shape[0],len(labels))), columns=labels, index= unique_sequences)
len(labels)

unique_sequences_freq_left=unique_sequences_freq_left.join(unique_sequences_freq_t)
unique_sequences_freq_left.index.names = ['Gene'] 
unique_sequences_freq_left.reset_index(level=0, inplace=True)
unique_sequences_freq_left.to_csv("/Users/moradigd/Documents/Plasmid/pan_genome_PB_1.csv", index=False)

        

In [ ]:
unique_sequences_freq_left

In [ ]:
np.empty((unique_sequences_freq_t.shape[0],len(labels)))

In [ ]:
import numpy as np
for k in range(2,4):
    import pandas as pd
    df_input=pd.read_csv("/Users/moradigd/Documents/Plasmid/input_agg.csv")
    
    df_input=df_input.iloc[:,[0,k]]
    
    df_input=df_input.dropna()
    df_input=df_input.reset_index()
    
    print(df_input.columns)
    print(df_input.shape)
    
    unique_sequences_set_agg= pd.DataFrame(index=np.arange(df_input.shape[0]))
    unique_sequences_agg=[]
    
    for g in range(5,20,2):
        print(g)
        unique_sequences_set=[]
#for i in range(len(df_input['seq'])):
        for i in range(df_input.shape[0]):
            unique_sequences_set=unique_sequences_set+getKmers(df_input['seq'][i], g, 1)
        unique_sequences=list(set(unique_sequences_set))
        unique_sequences_agg=unique_sequences_agg+unique_sequences

        [df_input['seq'][1].count(i) for i in unique_sequences]

        unique_sequences_freq=[]
#for i in range(len(df_input['seq'])):
        for j in range(df_input.shape[0]):
            unique_sequences_freq.append([df_input['seq'][j].count(i) for i in unique_sequences])
        unique_sequences_freq   
        unique_sequences_freq=pd.DataFrame(unique_sequences_freq)
#unique_sequences_freq.columns=unique_sequences
        #unique_sequences_freq.columns=['g_'+str(g)+'_'+str(x) for x in unique_sequences_freq.columns]
        unique_sequences_freq.columns=unique_sequences
        unique_sequences_set_agg=unique_sequences_set_agg.join(unique_sequences_freq)
        print(unique_sequences_set_agg.shape)
        
    unique_sequences_freq_t=unique_sequences_set_agg.transpose()
    unique_sequences_freq_t.columns=['seq'+str(i) for i in range(unique_sequences_freq_t.shape[1])]
    unique_sequences_freq_t.index=unique_sequences_agg
        
        
    labels=["Non-unique Gene name","Annotation","No. isolates","No. sequences","Avg sequences per isolate","Genome Fragment","Order within Fragment","Accessory Fragment","Accessory Order with Fragment","QC","Min group size nuc","Max group size nuc","Avg group size nuc"]
    unique_sequences_freq_left=pd.DataFrame(np.zeros((unique_sequences_freq_t.shape[0],len(labels))), columns=labels, index= unique_sequences_agg)
    len(labels)

    unique_sequences_freq_left=unique_sequences_freq_left.join(unique_sequences_freq_t)
    unique_sequences_freq_left.index.names = ['Gene'] 
    unique_sequences_freq_left.reset_index(level=0, inplace=True)
    print(unique_sequences_freq_left.shape)
    unique_sequences_freq_left.to_csv("/Users/moradigd/Documents/Plasmid/pan_genome_agg_"+df_input.columns[2]+".csv", index=False)


In [ ]:
unique_sequences_set_agg.shape

In [ ]:
len(unique_sequences_agg)

In [ ]:
unique_sequences_freq_t=unique_sequences_set_agg.transpose()

In [ ]:
unique_sequences_freq_t.columns=['seq'+str(i) for i in range(unique_sequences_freq_t.shape[1])]

In [ ]:
def processor(label="labels"):
    file_input=open('/Users/moradigd/Documents/streamlit/input.fasta', 'r')
    labels=[]
    sequences=[]

    for i in file_input.readlines():
        if ">" in i:
            labels.append(i.replace(">", "").replace('\n', ''))
        else:
            sequences.append(i.replace('\n', ''))
    if label=="labels":
        return labels
    elif label=="sequence":
        return sequences
processor(label="sequence")
 

In [ ]:
labels=["Non-unique Gene name","Annotation","No. isolates","No. sequences","Avg sequences per isolate","Genome Fragment","Order within Fragment","Accessory Fragment","Accessory Order with Fragment","QC","Min group size nuc","Max group size nuc","Avg group size nuc"]
unique_sequences_freq_left=pd.DataFrame(np.zeros((unique_sequences_freq_t.shape[0],len(labels))), columns=labels, index= unique_sequences_agg)
len(labels)

#unique_sequences_freq_t.index=unique_sequences

unique_sequences_freq_left=unique_sequences_freq_left.join(unique_sequences_freq_t)
unique_sequences_freq_left.index.names = ['Gene'] 
unique_sequences_freq_left.reset_index(level=0, inplace=True)
unique_sequences_freq_left.to_csv("/Users/moradigd/Documents/Plasmid/pan_genome_PB_1.csv", index=False)


In [ ]:
unique_sequences_freq_t

In [ ]:
df_input.columns[2]

In [ ]:
unique_sequences_freq_left.to_csv("/Users/moradigd/Documents/Plasmid/pan_genome_"+df_input.columns[2]+".csv", index=False)


In [ ]:
unique_sequences_freq_left=unique_sequences_freq_left.join(unique_sequences_freq_t)

In [ ]:
#Isolation forests morphology
import pandas as pd
input_df=pd.read_csv("/Users/moradigd/Documents/chemicalgenomic/similarity_morphology.csv", index_col=0)
input_df




In [ ]:
input_df.head
input_df_red=input_df

from sklearn.ensemble import IsolationForest
clf=IsolationForest(n_estimators=100, max_samples='auto', contamination=float(.12), max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42, verbose=0)
clf.fit(input_df_red)
pred = clf.predict(input_df_red)
pred


input_df_red['anomaly']=pred
outliers=input_df_red.loc[input_df_red['anomaly']==-1]
outlier_index=list(outliers.index)
#print(outlier_index)
#Find the number of anomalies and normal points here points classified -1 are anomalous
print(input_df_red['anomaly'].value_counts())




In [ ]:
from sklearn.ensemble import IsolationForest
import pandas as pd
import numpy as np
input_df=pd.read_csv("/Users/moradigd/Documents/chemicalgenomic/similarity_colony_circularity.csv")
input_df

range_cont=np.arange(0.01,0.5,0.05)
for i in range(len(range_cont)):
    import pandas as pd
    input_df=pd.read_csv("/Users/moradigd/Documents/chemicalgenomic/similarity_colony_circularity.csv")
    input_df_red=input_df
    print(i)
    clf=IsolationForest(n_estimators=100, max_samples='auto', contamination=float(range_cont[i]), max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42, verbose=0)
    clf.fit(input_df_red)
    pred = clf.predict(input_df_red)

    input_df_red['anomaly']=pred
    outliers=input_df_red.loc[input_df_red['anomaly']==-1]
    outlier_index=list(outliers.index)
    print(input_df_red['anomaly'].value_counts())
    pd.DataFrame({"names":input_df_red.columns[:-1], "anomaly":pred}).to_csv("/Users/moradigd/Documents/chemicalgenomic/anomaly_circularity_labels"+str(i)+".csv")


In [ ]:
import numpy 
numpy.version.version

In [ ]:

pd.DataFrame({"names":input_df_red.columns[:-1], "anomaly":pred}).to_csv("/Users/moradigd/Documents/chemicalgenomic/anomaly_labels.csv")


In [ ]:
np.arange(0.01,0.5,0.05)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D
pca = PCA(n_components=3)  # Reduce to k=3 dimensions
scaler = StandardScaler()

to_model_columns=input_df_red.columns[1:13]
#normalize the metrics
X = scaler.fit_transform(input_df_red[to_model_columns])
X_reduce = pca.fit_transform(X)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_zlabel("x_composite_3")
# Plot the compressed data points
ax.scatter(X_reduce[:, 0], X_reduce[:, 1], zs=X_reduce[:, 2], s=4, lw=1, label="inliers",c="green")
# Plot x's for the ground truth outliers
ax.scatter(X_reduce[outlier_index,0],X_reduce[outlier_index,1], X_reduce[outlier_index,2],
           lw=2, s=60, marker="x", c="red", label="outliers")
ax.legend()
plt.show()

In [ ]:
import plotly.express as px


In [ ]:

from sklearn.decomposition import PCA

df = input_df_red

pca = PCA(n_components=2)
components = pca.fit_transform(input_df_red)
labels = input_df_red['anomaly']

fig = px.scatter(components, x=0, y=1, color=labels)
fig.show()

In [ ]:


exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)

px.area(
    x=range(1, exp_var_cumul.shape[0] + 1),
    y=exp_var_cumul,
    labels={"x": "# Components", "y": "Explained Variance"}
)

In [ ]:
import numpy as np
input_df=pd.read_csv("/Users/moradigd/Documents/chemicalgenomic/similarity_morphology_matrix.csv")
input_df
input_df_red=input_df

from sklearn.ensemble import IsolationForest
clf=IsolationForest(n_estimators=100, max_samples='auto', contamination=float(.5), max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42, verbose=0)
clf.fit(input_df_red)
pred = clf.predict(input_df_red)
pred


input_df_red['anomaly']=pred
outliers=input_df_red.loc[input_df_red['anomaly']==-1]
outlier_index=list(outliers.index)

import time
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=20, n_iter=300)
tsne_results = tsne.fit_transform(input_df_red)
print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))


import seaborn as sns
import matplotlib.pyplot as plt

input_df_red['tsne-2d-one'] = tsne_results[:,0]
input_df_red['tsne-2d-two'] = tsne_results[:,1]
input_df_red['y'] =input_df_red['anomaly']
plt.figure(figsize=(16,10))
scatter=sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue="y",
    palette=sns.color_palette("hls", 2),
    data=input_df_red,
    legend="full",
    alpha=0.3
)

scatter.set_xlim(left=-35, right=35)
scatter.set_ylim(bottom=-35, top=35)



In [ ]:
import time
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(input_df_red)
print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

input_df_red['tsne-2d-one'] = tsne_results[:,0]
input_df_red['tsne-2d-two'] = tsne_results[:,1]
input_df_red['y'] =input_df_red['anomaly']
plt.figure(figsize=(16,10))
scatter=sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue="y",
    palette=sns.color_palette("hls", 2),
    data=input_df_red,
    legend="full",
    alpha=0.3
)

scatter.set_xlim(left=-35, right=35)
scatter.set_ylim(bottom=-35, top=35)

In [ ]:
import Image
import cv2
img1 = cv.imread('ml.png')
img2 = cv.imread('opencv-logo.png')
dst = cv.addWeighted(img1,0.7,img2,0.3,0)
cv.imshow('dst',dst)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
import cv2
img1 = cv2.imread('/Users/moradigd/Documents/chemicalgenomic/morphology/image_output_PA14_18630.jpg')
img2 = cv2.imread('/Users/moradigd/Documents/chemicalgenomic/morphology/image_output_PA14_18640.jpg')
dst = cv2.addWeighted(img1,0.5,img2,0.5,0)
cv2.imshow('dst',dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2
files=glob.glob("/Users/moradigd/Documents/chemicalgenomic/morphology/test/*")
img1 = cv2.imread(files[0])
img2 = cv2.imread(files[1])

dst = cv2.addWeighted(img1,0.5,img2,0.5,0)

for i in range(2,4):
    img_tmp = cv2.imread(files[i])
    dst = cv2.addWeighted(dst,i/(i+1),img_tmp,1/(i+1),0)
    print(i)

import pandas as pd
df=pd.read_csv("/Users/moradigd/Documents/chemicalgenomic/df_clustering_similarity_morphology_shortened.csv")
isolates=df.isolate[df.cluster==6].values
isolates_list=["/Users/moradigd/Documents/chemicalgenomic/morphology/image_output_"+i+".jpg" for i in isolates]
isolates_list

In [ ]:
import cv2
import pandas as pd
df=pd.read_csv("/Users/moradigd/Documents/chemicalgenomic/df_clustering_similarity_colour.csv")
isolates=df.isolate[df.cluster==9].values
files=["/Users/moradigd/Documents/chemicalgenomic/morphology/image_output_"+i+".jpg" for i in isolates]

#files=glob.glob("/Users/moradigd/Documents/chemicalgenomic/morphology/test/*")

img1 = cv2.imread(files[0])
img2 = cv2.imread(files[1])

dst = cv2.addWeighted(img1,0.5,img2,0.5,0)

#for i in range(2,len(files)):
for i in range(2,6):
    print(i)
    img_tmp = cv2.imread(files[i])
    dst = cv2.addWeighted(dst,i/(i+1),img_tmp,1/(i+1),0)

cv2.imshow('dst',dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

2
3
4
5


In [ ]:
cv2.imread(files[1])
i

In [ ]:
cv2.addWeighted(dst,i/(i+1),img_tmp,1/(i+1),0)